## basics

- policy function: $\pi(a|s)$ is a PDF
    - $\sum_{a}\pi(a|s)=1$
- Policy Network: $\pi(a|s;\mathbf \theta)$
    - $\bf\theta$ 表示神经网络的待训练参数
    - $\sum_{a}\pi(a|s;\theta)=1$（last layer of nn，称之为 logits，接一个 softmax 的变换）
- 状态价值函数（state value function），对其做函数近似（function approximation）；

    - **Discounted reward**

        $$
        U_t=R_t+\gamma R_{t+1} + \gamma^2R_{t+2} + \gamma^3R_{t+3}+\cdots
        $$

        - $U_t$ 的精确计算依赖 $A_t,A_{t+1},A_{t+2}, \cdots, $ 和 $S_t,S_{t+1},S_{t+2},\cdots$
            - 是未来所有奖励的加和
        - $t$ 时刻，未来的 reward 为还未观测到的随机变量，
            - 每个奖励 $R_t$ 的随机性都来自于前一个时刻的动作 $A_t$ 和 $S_t$
            - 动作的随机性来自于策略函数 $\pi(a|\cdot)$，状态的随机性来自于状态转移函数 $p(s_t|s_{t-1})$
                - 随机性意味着它们（$A_t,S_t$）都是随机变量


### action-value function 与 state-value function

- **action-value function**（通过条件期望来定义）
    $$
    Q_\pi(s_t,a_t)=\mathbb E[U_t|S_t=s_t,A_t=a_t]
    $$
    - $Q_\pi(s_t,a_t)$ 用来评价在状态 $S_t=a_t$ 下选择动作 $a_t$ 的好坏程度
- **state-value function**（是 $Q_\pi$ 的期望，将 $Q_\pi$ 中的动作 $A\sim \pi(\cdot|s_t)$ 积分掉）

    $$
    V_\pi(s_t)=\mathbb E_A\left[Q_\pi(s_t,A)\right]
    $$
    
    - $V_\pi(s_t)$ 越大，说明当前状态 $s_t$ 的胜算越大；
    - 如果 $A\sim \pi(\cdot|s_t)$ 是离散的，则有：
    
    $$
    V_\pi(s_t)=\mathbb E_A\left[Q_\pi(s_t,A)\right]=\sum_a\pi(a|s_t)Q_\pi(s_t,a)
    $$

### 策略学习的目标函数

- $S_t,A_t,S_{t+1},A_{t+1}, S_{t+2}, A_{t+2}$
- $Q_\pi(s_t,a_t)=\mathbb E[U_t|S_t=s_t,A_t=a_t]$
- $V_\pi(s_t;\theta)=\mathbb E_{A_t\sim \pi(\cdot|s_t;\theta)}Q_\pi(s_t,A_t)=\sum_a\pi(a|s;\theta)Q_\pi(s,a)$
- 状态价值（state value）既依赖当前状态 $s_t$ 也依赖策略网络 $\pi$ 的参数 $\theta$
    - 当前状态 $s_t$ 越好，$V_\pi(s_t)$ 越大，回报 $U_t$ 的期望也就越大；
    - 策略网络 $\pi$ 越好，$V_\pi(s_t)$ 也会越大；
- 定义目标函数

    $$
    \begin{split}
    &J(\theta)=\mathbb E_S[V_\pi(S)]\\
    &\max_{\theta} J(\theta)
    \end{split}
    $$
    
    - 最大化目标，通过梯度上升来优化
    
    $$
    \theta\leftarrow \theta+\eta \frac{\partial J(\theta)}{\partial \theta}
    $$

### 策略梯度（policy gradient）

$$
\theta\leftarrow \theta+\eta \frac{\partial V(s;\theta)}{\partial \theta}
$$

- 策略梯度不是真正的梯度，真正的梯度是 $\frac{\partial J(\theta)}{\partial \theta}$
- 而 state-value function $V(s;\theta)$ 关于 $\theta$ 的导数称之为 policy gradient
    - 其实是一个随机梯度，随机性来源于 $s$

### 策略梯度定理

$$V_\pi(s_t;\theta)=\mathbb E_{A_t\sim \pi(\cdot|s_t;\theta)}Q_\pi(s_t,A_t)=\sum_a\pi(a|s;\theta)Q_\pi(s,a)$$

$$
\begin{split}
&
\frac{\partial V(s;\theta)}{\partial \theta}\approx \sum_a \frac{\partial \pi(a|s;\theta)}{\partial \theta}Q_\pi(s,a)\\
&
\frac{\partial V(s;\theta)}{\partial \theta}\approx\mathbb E_{A\sim \pi(\cdot|S;\theta)}\left[\frac{\partial \ln_\pi(A|S;\theta)}{\partial \theta}\cdot Q_\pi(S,A)\right]
\\
&
\frac{\partial J(\theta)}{\partial \theta}\approx\mathbb E_S\left[\mathbb E_{A\sim \pi(\cdot|S;\theta)}\left[\frac{\partial \ln_\pi(A|S;\theta)}{\partial \theta}\cdot Q_\pi(S,A)\right]\right]
\end{split} 
$$

#### 离散型计算示例

- 采用第一个公式
- 记 $f(a,\theta)=\frac{\partial \pi(a|s;\theta)}{\partial \theta}Q_\pi(s,a)$
    - 然后枚举所有的离散型动作 $a$，计算 $\frac{\partial V(s;\theta)}{\partial \theta}=\sum_af(a,\theta)$

#### 连续型

$$
\frac{\partial V(s;\theta)}{\partial \theta}\approx\mathbb E_{A\sim \pi(\cdot|S;\theta)}\left[\frac{\partial \ln_\pi(A|S;\theta)}{\partial \theta}\cdot Q_\pi(S,A)\right]
$$

- $\pi(\cdot|S;\theta)$ 是一个神经网络，难以直接求定积分
- 只好做蒙特卡洛近似，将期望近似算出来
    - sample $\hat a\sim \pi(\cdot|s;\mathbf \theta)$
    - 计算 $g(\hat a, \theta)=\frac{\partial \ln_\pi(\hat a|s;\theta)}{\partial \theta}\cdot Q_\pi(s,\hat a)$
        - 基于 pytorch 等深度学习框架的自动求导机制；

### 策略梯度 algorithm

1. observe the state $s_t$
2. sample $a_t \sim \pi(\cdot|s_t;\theta_t)$
3. compute $q_t\approx Q_\pi(s_t,a_t)$ ？？
4. differentiate policy network $d_{\theta,t}=\frac{\partial \log\pi(a_t|s_t;\theta)}{\partial \theta}\big|_{\theta=\theta_t}$
5. **policy gradient**: $g(a_t,\theta_t)=q_td_{\theta,t}$
6. update policy network: $\theta_{t+1}=\theta_t+\beta\cdot g(a_t,\theta_t)$

#### 计算 $Q_\pi(s_t,a_t)$ 的 REINFORCE

- trajectory：$(s_1,a_1,r_1), (s_2, a_2, r_2),\cdots, (s_T, a_T, r_T )$
- compute discounted return $u_t=\sum_{k=t}^T\gamma^{k-t}r_k, \forall t$
- since $Q_\pi(s_t,a_t)=E[U_t]$, use $u_t$ approximate $Q_\pi(s_t,a_t)$
- $q_t\approx Q_\pi(s_t,a_t)\approx u_t$

## REINFOCE with baseline

## torch 实现 REINFORCE with baseline

In [5]:
import torch
from torch import nn
import torch.nn.functional as F

In [9]:
class PolicyNet(nn.Module):
    '''输入是状态，返回是action的概率分布'''
    def __init__(self, num_state, num_action):
        super().__init__()
        self.fc1 = nn.Linear(num_state, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_action)
    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        prob = F.softmax(x, dim=-1)
        return prob

In [11]:
class ValueNet(nn.Module):
    '''输入是状态，返回是 value（一个scalar）'''
    def __init__(self, num_state):
        super().__init__()
        self.fc1 = nn.Linear(num_state, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
    def forward(self, state):
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(state))
        x = self.fc3(state)
        return x